In [1]:
import xarray as xr
import numpy as np
import dask.array as da
import matplotlib.pyplot as plt
import os
from dask.distributed import Client, LocalCluster
from datetime import datetime,timedelta
import glob
import indices_function as ifun
import warnings
from rechunker import rechunk

warnings.filterwarnings('ignore')

In [2]:
cluster = LocalCluster(
    n_workers=20, 
    threads_per_worker=1,
    timeout='8600s',
    memory_limit='150GB',   
)
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 20
Total threads: 20,Total memory: 2.73 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:32997,Workers: 20
Dashboard: http://127.0.0.1:8787/status,Total threads: 20
Started: Just now,Total memory: 2.73 TiB
Comm: tcp://127.0.0.1:38941,Total threads: 1
Dashboard: http://127.0.0.1:37779/status,Memory: 139.70 GiB
Nanny: tcp://127.0.0.1:34445,


In [13]:
# import datatree
variables=['tasmax']
for var in variables:
    datasets=[]
# List of file names
    for year in range(2050,2055):
        if year==1999:
            continue
        X1=xr.open_zarr("/nobackupp28/skhajehe/dcp-indices/multimodel/"+var+"/"+str(year)+".zarr",consolidated=False)
        datasets.append(X1)
    combined_data = xr.concat(datasets, dim='time')
    pr_annual_proj=combined_data.chunk(dict(time=-1,lat=200,lon=200)).groupby('time.season').mean(dim='time')
    pr_annual_proj.to_zarr("/nobackupp28/skhajehe/dcp-indices/multimodel/seasonal_average/"+var+"/2050.zarr",mode="w")

### Annual Mean for DCP30 and PRISM

In [9]:
# import datatree
variables=['vpr','pr','tasmax','tasmin']
for var in variables[:1]:
    datasets=[]
# List of file names
    for year in range(1984,2015):
        if year==1999:
            continue
        X1=xr.open_zarr("/nobackupp10/skhajehe/dcp-indices/multimodel/"+var+"/"+str(year)+".zarr",consolidated=False)
        datasets.append(X1)
    combined_data = xr.concat(datasets, dim='time')
    pr_annual_proj=combined_data.chunk(dict(time=-1,lat=200,lon=200)).groupby('time.year').mean(dim='time').mean(dim="year")
    pr_annual_proj.to_zarr("/nobackupp10/skhajehe/dcp-indices/multimodel/annual_average/"+var+"/2014.zarr",mode="w")

In [8]:
# import datatree
variables=['vpr','pr','tasmax','tasmin']
for var in variables[:1]:
    datasets=[]
    netcdf_list=glob.glob("/nobackupp10/nexprojects/GDDP2/obs/prism/day/"+var+"/*.nc")
    netcdf_list.sort()
    combined_data=xr.open_mfdataset(netcdf_list[3:])
    pr_annual_proj=combined_data.chunk(dict(time=-1,lat=200,lon=200)).groupby('time.year').mean(dim='time').mean(dim="year")
    pr_annual_proj.to_zarr("/nobackupp10/skhajehe/dcp-indices/prism/annual_average/"+var+"/2014.zarr",mode="w")